In [1]:
import os
import numpy as np
import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing import image
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from keras_vggface.vggface import VGGFace

Using TensorFlow backend.


In [2]:
vggface_model = VGGFace(
    include_top=True,
    input_shape=(224, 224, 3),
    weights='vggface')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
#vggface_model.summary()

In [4]:
vggface_model = VGGFace(
    include_top=False,
    input_shape=(224, 224, 3),
    weights='vggface')

vggface_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [5]:
custom_vggface_model = keras.models.Sequential()
custom_vggface_model.add(vggface_model)
custom_vggface_model.add(layers.Flatten())
custom_vggface_model.add(layers.Dense(512, activation='relu', name='fc6'))
custom_vggface_model.add(layers.Dense(512, activation='relu', name='fc7'))
custom_vggface_model.add(layers.Dense(4, activation='softmax', name='fc8'))

In [6]:
custom_vggface_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vggface_vgg16 (Model)        (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
fc6 (Dense)                  (None, 512)               12845568  
_________________________________________________________________
fc7 (Dense)                  (None, 512)               262656    
_________________________________________________________________
fc8 (Dense)                  (None, 4)                 2052      
Total params: 27,824,964
Trainable params: 27,824,964
Non-trainable params: 0
_________________________________________________________________


In [7]:
print('This is the number of trainable weights before freezing the conv base:', 
      len(custom_vggface_model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 32


In [8]:
vggface_model.trainable = False

In [9]:
print('This is the number of trainable weights after freezing the conv base:', 
      len(custom_vggface_model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 6


In [10]:
base_dir = '/Users/mvpzhao/data/vgg-face/faces_cut'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 224x224
        target_size=(224, 224),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

custom_vggface_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = custom_vggface_model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=1)

Found 36 images belonging to 4 classes.
Found 16 images belonging to 4 classes.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/30
  2/100 [..............................] - ETA: 10:12 - loss: 1.0978 - acc: 0.7969 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00

In [21]:
from keras.preprocessing import image
from keras_vggface import utils

img = image.load_img('/Users/mvpzhao/Downloads/cps.png', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = utils.preprocess_input(x, version=1)

preds = custom_vggface_model.predict(x)
print('Predicted:', preds)

Predicted: [[7.3508527e-29 0.0000000e+00 0.0000000e+00 1.0000000e+00]]
